In [8]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import datetime as dt
import math
THIS_FOLDER = os.path.dirname(os.path.abspath("RPP-SBC-EEMS"))
print(THIS_FOLDER)

C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS


In [9]:
arcpy.ResetEnvironments()
gdb_name = "Summarize_Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
if arcpy.Exists(gdb_name):
    print("Already Exists")
else:
    try:
        arcpy.CreateFileGDB_management("./", gdb_name)
    except:
        print("{} already exists".format(gdb_name))
        pass
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

Already Exists


In [10]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pr_Fire_Ign.shp",
        "in_data/Pct_Cg_Lg_Fi.shp", "in_data/Pct_Cg_Fi_Ign.shp",
        "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp",
        "in_data/FloraFaunaMerge.shp",
        "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units.shp",
        "out_data/building/Building_Intersect.shp",
        "in_data/Slope.shp", "in_data/Veg_Only.shp",
        "in_data/FHSZ.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

In [11]:
arcpy.analysis.Intersect("Reporting_Units_rpj; Pr_Fire_Ign_rpj", "A_RU_0_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_0_int", "Pr_Lg_Fire_rpj", 
                               "A_RU_1", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_1
1,


In [12]:
arcpy.analysis.Intersect("A_RU_1; Pr_Fire_Ign_rpj", "A_RU_1_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_1_int", "Pr_Fire_Ign_rpj", 
                               "A_RU_2", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_2
1,


In [13]:
arcpy.analysis.Intersect("A_RU_2; Pct_Cg_Lg_Fi_rpj", "A_RU_2_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_2_int", "Pct_Cg_Lg_Fi_rpj", 
                               "A_RU_3", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_3
1,


In [14]:
arcpy.analysis.Intersect("A_RU_3; Pct_Cg_Fi_Ign_rpj", "A_RU_3_int", "ALL", None, "INPUT")
arcpy.analysis.SummarizeWithin("A_RU_3_int", "Pct_Cg_Fi_Ign_rpj", 
                               "A_RU_4", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_4
1,


In [18]:
# arcpy.analysis.Intersect("A_RU_4; WUI_Intermix_rpj", "A_RU_4_int", "ALL", None, "INPUT")
arcpy.SummarizeWithin_analysis("WUI_Intermix_rpj", "A_RU_4", 
                               "A_RU_5", "KEEP_ALL", "SHAPE_AREA mean", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_5
1,


In [19]:
# arcpy.analysis.Intersect("A_RU_5; WUI_Interface_rpj", "A_RU_5_int", "ALL", None, "INPUT")
arcpy.analysis.SummarizeWithin("A_RU_5", "WUI_Interface_rpj", 
                               "A_RU_6", "KEEP_ALL", "SHAPE_AREA mean", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_6
1,


In [22]:
arcpy.analysis.Intersect("A_RU_6; LU_Merged_rpj", "A_RU_6_int", "ALL", None, "INPUT")
arcpy.analysis.SummarizeWithin("A_RU_6_int", "LU_Merged_rpj", 
                               "A_RU_7", "KEEP_ALL", "LU_Cat Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "LU_Cat", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_7
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\LU_Cat_Summary


In [23]:
arcpy.analysis.Intersect("A_RU_7; FloraFaunaMerge_rpj", "A_RU_7_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_7_int", "FloraFaunaMerge_rpj", 
                               "A_RU_8", "KEEP_ALL", "Flora_Faun Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_8
1,


In [24]:
# arcpy.analysis.Intersect("A_RU_8; Building_Intersect_rpj", "A_RU_8_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_8", "Building_Intersect_rpj", 
                               "A_RU_9", "KEEP_ALL", "", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_9
1,


In [25]:
arcpy.analysis.Intersect("A_RU_9; Slope_rpj", "A_RU_9_int", "ALL", None, "INPUT")


arcpy.analysis.SummarizeWithin("A_RU_9_int", "Slope_rpj", 
                               "A_RU_10", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_10
1,


In [26]:
arcpy.analysis.Intersect("A_RU_10; Veg_Only_rpj", "A_RU_10_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_10_int", "Veg_Only_rpj", 
                               "A_RU_11", "KEEP_ALL", "gridcode Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "gridcode", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_11
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\gridcode_Summary


In [27]:
arcpy.analysis.Intersect("A_RU_11; FHSZ_rpj", "A_RU_11_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_11_int", "FHSZ_rpj", 
                               "A_RU_12", "KEEP_ALL", "HAZ_CODE Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "HAZ_CODE", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_12
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\HAZ_CODE_Summary


In [30]:
arcpy.AddField_management("A_RU_12", "WUI_Interm", "LONG")
arcpy.AddField_management("A_RU_12", "WUI_Interf", "LONG")

codeblock = """
def get_val(m):
    if m > 0:
        return 1
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "WUI_Interm", "get_val(!SUM_Area_SQUAREFEET!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("A_RU_12", "WUI_Interf", "get_val(!SUM_Area_SQUAREFEET_1!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET', 'SUM_Area_SQUAREFEET_1'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_12'>

In [31]:
arcpy.AddField_management("A_RU_12", "Mj_LU_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_LU_Cat", "get_val(!Majority_LU_Cat!, !Majority_LU_Cat_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_LU_Cat', 'Minority_LU_Cat', 'Majority_LU_Cat', 'Majority_LU_Cat_Percent', 'Minority_LU_Cat_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_12'>

In [32]:
arcpy.AddField_management("A_RU_12", "Mj_Haz_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_Haz_Cat", "get_val(!Majority_HAZ_CODE!, !Majority_HAZ_CODE_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_HAZ_CODE', 'Minority_HAZ_CODE', 'Majority_HAZ_CODE', 'Majority_HAZ_CODE_Percent', 'Minority_HAZ_CODE_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_12'>

In [33]:
arcpy.AddField_management("A_RU_12", "Vg_Pct", "FLOAT")

codeblock = """
def get_val(a, m):
    return round(m/100, 2)
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Vg_Pct", "get_val(!Majority_gridcode!, !Majority_gridcode_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_gridcode', 'Minority_gridcode', 'Minority_gridcode_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_12'>

In [35]:
fieldObs = arcpy.ListFields("A_RU_12")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_12", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

print(fieldNames)

['OBJECTID', 'Shape', 'FID_A_RU_11', 'FID_A_RU_10', 'FID_A_RU_9', 'FID_A_RU_7', 'FID_A_RU_6', 'WUI_DESC', 'Shape_Leng', 'mean_shape_area', 'Polygon_Count', 'mean_shape_area_1', 'Polygon_Count_1', 'FID_LU_Merged_rpj', 'OBJECTID_1', 'Shape_Leng_1', 'golf', 'ag', 'lake', 'desert', 'LU_Cat', 'sum_Area_SQUAREFEET_12', 'Polygon_Count_12', 'Join_ID', 'FID_FloraFaunaMerge_rpj', 'Amphibian_', 'Amphibian1', 'Birds_Suit', 'Birds_Su_1', 'CNDDB', 'CNDDB_Raw', 'CNDDB_S1', 'CNDDB_S1_R', 'CNDDB_S2', 'CNDDB_S2_R', 'CNDDB_S3', 'CNDDB_S3_R', 'CNDDB_S4', 'CNDDB_S4_R', 'Connectivi', 'Cores', 'Cores_Raw', 'Corridors', 'Corridors_', 'Critical_H', 'Critical_1', 'FFAreaOfIm', 'Fauna_Suit', 'Fauna_Su_1', 'Flora_Faun', 'Gap_Analys', 'Gap_Anal_1', 'Mammal_Sui', 'Mammal_S_1', 'Rare_Threa', 'Rare_Thr_1', 'Reptile_Su', 'Reptile__1', 'Steelhead_', 'Steelhead1', 'Wetlands', 'Wetlands_R', 'unique_id', 'CSVID', 'Shape_Leng_12', 'Shape_Le_1', 'mean_flora_faun', 'Polygon_Count_12_13', 'sum_Area_SQUAREFEET_12_13', 'Polygon

In [34]:
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_1', 'Pr_Fi_Ign', 'Pr_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12', 'Pc_Cg_LgFi', 'Pc_Cg_LgFi')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13', 'Pc_Fi_Ign', 'Pc_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13_14', 'Slp', 'Slp')

arcpy.AlterField_management('A_RU_12', 
                            'MEAN_Flora_Faun', 'Fl_Fa', 'Fl_Fa')

arcpy.AlterField_management('A_RU_12', 
                            'SUM_Area_SQUAREFEET_12_13', 'Buil_Ar', 'Buil_Ar')

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET_12', 'Join_ID_1', 'Shape_Leng', 'SUM_Area_SQUAREFEET_12_13_14', 
                              'SUM_Area_SQUAREFEET_12_13_14_15', "Majority_gridcode", "Majority_gridcode_Percent"])

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000728: Field MEAN_gridcode_1 does not exist within table
Failed to execute (AlterField).


In [ ]:
arcpy.AddField_management("A_RU_12", "Buil_Rat", "FLOAT")

arcpy.CalculateField_management("A_RU_12", "Buil_Rat", 
                                '!Buil_Ar!/!Shape_Area!', "PYTHON3")

In [ ]:
fieldObs = arcpy.ListFields("A_RU_12") 

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

In [ ]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_12", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

In [ ]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.shp"))
gdf.info()

In [ ]:
gdf.to_csv("out_data/eems/A_RU_12.csv", index=False)